This notebook contains some HDF5-manipulation code to write a new dataset `leftright` to a set of training samples. The `leftright` dataset should contain the joint coordinates for the left *or* right arm present in the sample; this means that both left arms and right arms will have a nonzero `leftright` entry.

The aim here is to get a regressor which regresses any arms in an image, regardless of whether they're left or right. Presumably I'll still need the left/right classifier to do its thing as well if I'm to have any hope of integrating this into a real pipeline.

In [ ]:
import h5py
import numpy as np

In [ ]:
h5_paths = [
    '../cache/train-patches-mpii/samples-000001.h5',
    '../cache/train-patches-mpii/negatives.h5',
    '../cache/val-patches-mpii/samples-000001.h5',
    '../cache/val-patches-mpii/negatives.h5'
]

def add_leftright(path, overwrite=False):
    with h5py.File(path, 'r+') as fp:
        # Overwrite check
        if 'leftright' in fp.keys():
            print('Warning: "{}" already has a leftright dataset'.format(path))
            if not overwrite:
                print('Skipping')
                return
            print('Overwriting it')
            del fp['leftright']
            
        # We'll save these old values for some assertions later
        old_l = fp['left'][:]
        old_r = fp['right'][:]
        
        # Now we can write the dataset
        leftright = np.array(fp['left'][:])
        class_nums = np.argmax(fp['class'], axis=1)
        left_mask = class_nums == 1
        right_mask = class_nums == 2
        # Make sure we don't have any real annotations in there already
        assert (leftright[right_mask] == 0).all()
        # I'm doing [:] because there's a bug in h5py's selections.py;
        # specifically, FancySelection.__getitem__ relies on a count
        # variable which is zero when the selection vector is empty
        # (or something). I should probably report this to the devs.
        leftright[right_mask] = fp['right'][:][right_mask, :]
        fp['leftright'] = leftright
        
        # Make sure that our new leftright dataset corresponds to the
        # old left dataset and old right dataset
        new_lr = fp['leftright'][:]
        assert np.all(new_lr[right_mask] == old_r[right_mask])
        assert np.all(new_lr[left_mask] == old_l[left_mask])

for path in h5_paths:
    print('Adding leftright to {}'.format(path))
    add_leftright(path)